In [1]:
!pip install huggingface_hub --quiet
from huggingface_hub import login
login()

In [4]:
!pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install torch==2.7.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 50.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 66.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 66.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 59.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 66.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 66.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 60.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 66.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from pathlib import Path
import os, csv, time, json
from PIL import Image
import torch

BASE_DIR   = Path.cwd()
DATA_DIR   = BASE_DIR / "data"
IMAGES_DIR = DATA_DIR / "images"
ANNOT_CSV  = DATA_DIR / "annotations.csv"
LOGS_DIR   = BASE_DIR / "logs"
LOGS_DIR.mkdir(exist_ok=True)

assert DATA_DIR.exists(),   f"Не нашёл {DATA_DIR}"
assert IMAGES_DIR.exists(), f"Не нашёл {IMAGES_DIR}"
assert ANNOT_CSV.exists(),  f"Не нашёл {ANNOT_CSV}"

with open(ANNOT_CSV, encoding="utf-8", newline="") as fin:
    sample = fin.read(2048)
    fin.seek(0)
    dialect = csv.Sniffer().sniff(sample, delimiters=[",", ";", "\t"])
    reader = csv.DictReader(fin, dialect=dialect)
    image_field, ref_field = reader.fieldnames[:2]

print("✔ Пути настроены и CSV читается с delimiter=", dialect.delimiter)
print("Columns:", reader.fieldnames)

✔ Пути настроены и CSV читается с delimiter= ;
Columns: ['image_path', 'caption']


In [10]:
from transformers import (
    Blip2Processor, Blip2ForConditionalGeneration,
    MarianTokenizer, pipeline
)

MODEL_NAME = "Salesforce/blip2-flan-t5-xl"
TRANSLATOR = "Helsinki-NLP/opus-mt-en-ru"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

processor = Blip2Processor.from_pretrained(MODEL_NAME, use_auth_token=True)
model     = Blip2ForConditionalGeneration.from_pretrained(MODEL_NAME, use_auth_token=True).to(device)

tokenizer_mt = MarianTokenizer.from_pretrained(TRANSLATOR, use_auth_token=True)
translator   = pipeline(
    "translation_en_to_ru",
    model=TRANSLATOR,
    tokenizer=tokenizer_mt,
    device=0 if device=="cuda" else -1,
    framework="pt"
)

print("Модели загружены.")

Device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Модели загружены.


In [11]:
PROMPT = (
    "You are an AI assistant that generates visual descriptions for blind users. "
    "Describe the image in 2–3 clear, objective sentences, mentioning all key objects, "
    "their colors, shapes, textures, and spatial relationships, as well as any relevant "
    "background context. Avoid subjective or emotional language."
)

In [12]:
timestamp = time.strftime("%Y%m%d_%H%M%S")
log_path  = LOGS_DIR / f"run_notebook_{timestamp}.jsonl"
print("Logging to:", log_path)

with open(ANNOT_CSV, encoding="utf-8", newline="") as fin, \
     open(log_path, "w", encoding="utf-8") as fout:

    dialect = csv.Sniffer().sniff(fin.read(2048), delimiters=[",",";","\t"])
    fin.seek(0)
    reader = csv.DictReader(fin, dialect=dialect)
    image_field, ref_field = reader.fieldnames[:2]

    for row in reader:
        img_name = row[image_field]
        ref_text = row.get(ref_field, "")
        img_path = IMAGES_DIR / img_name

        if not img_path.exists():
            print(f"[WARN] нет файла: {img_name}")
            continue

        img    = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, text=PROMPT, return_tensors="pt").to(device)

        t0  = time.time()
        out = model.generate(**inputs, max_new_tokens=64, num_beams=5)
        t1  = time.time()
        en_cap = processor.decode(out[0], skip_special_tokens=True)

        t2     = time.time()
        ru_cap = translator(
            en_cap,
            max_length=128,
            clean_up_tokenization_spaces=True
        )[0]["translation_text"]
        t3     = time.time()

        rec = {
            "image":           img_name,
            "reference":       ref_text,
            "blip2_en":        en_cap,
            "ru_translated":   ru_cap,
            "latency_blip2_s": round(t1 - t0, 3),
            "latency_mt_s":    round(t3 - t2, 3)
        }
        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
        print(rec)

print("Лог сохранён в", log_path)

Logging to: /home/jovyan/work/tiflo_caption/logs/run_notebook_20250703_121538.jsonl
{'image': '1000092795.jpg', 'reference': 'Двое молодых парней с растрепанными волосами смотрят на свои руки, проводя время во дворе.', 'blip2_en': 'Two men are skateboarding in a garden', 'ru_translated': 'Двое мужчин скейтбордируют в саду.', 'latency_blip2_s': 0.496, 'latency_mt_s': 0.16}
{'image': '10002456.jpg', 'reference': 'Несколько мужчин в касках управляют гигантской подъемной системой.', 'blip2_en': 'a group of people are working on a crane', 'ru_translated': 'группа людей работает над краном', 'latency_blip2_s': 0.453, 'latency_mt_s': 0.055}
{'image': '1000268201.jpg', 'reference': 'Ребёнок в розовом платье поднимается по лестнице в прихожей.', 'blip2_en': 'a little girl is standing on the steps of a chicken coop', 'ru_translated': 'маленькая девочка стоит на ступеньках курятника', 'latency_blip2_s': 0.471, 'latency_mt_s': 0.098}
{'image': '1000344755.jpg', 'reference': 'Кто-то в синей рубашке

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'image': '1001573224.jpg', 'reference': 'Пятеро танцоров балета застыли на середине прыжка в танцевальной студии , в окно которой проникал солнечный свет.', 'blip2_en': 'a group of dancers in black leotards are performing in a dance studio', 'ru_translated': 'группа танцовщиц в чёрных леотардах выступает в танцевальной студии', 'latency_blip2_s': 0.611, 'latency_mt_s': 0.138}
{'image': '1001633352.jpg', 'reference': 'Трое молодых людей и молодая женщина в кроссовках подпрыгивают в воздухе на верхней площадке бетонной лестницы.', 'blip2_en': 'a group of people are jumping up and down stairs on a skateboard', 'ru_translated': 'группа людей прыгает вверх и вниз по лестнице на скейтборде', 'latency_blip2_s': 0.497, 'latency_mt_s': 0.124}
{'image': '1001773457.jpg', 'reference': 'Черная собака и белая с коричневыми пятнами собака смотрят друг на друга на улице.', 'blip2_en': 'two dogs are playing with a frisbee on the road', 'ru_translated': 'Две собаки играют с фрисби на дороге.', 'latenc